# Part 1 - Analysis Current Snapshot

In [8]:
import pandas as pd
import numpy as np
import datetime
import requests
import urllib.request
import time
import os
import math
from pathlib import Path
### Set TimeZone to HK if you are using docker container
#!rm /etc/localtime
#!ln -s /usr/share/zoneinfo/Asia/Hong_Kong /etc/localtime

### Load data from 資料一線通 (data.gov.hk)

The data source is "2019冠狀病毒病的本地最新情況(繁體中文)" / "Latest local situation of COVID-19 (Traditional Chinese)".

https://data.gov.hk/en-data/dataset/hk-dh-chpsebcddr-novel-infectious-agent/resource/3ab0040c-4272-4b9a-92bc-2a44b70fd3a4

The schema has changed over time, so there are at least two different schema

### Old schema (v1)

In [9]:
s = 'http://www.chp.gov.hk/files/misc/enhanced_sur_covid_19_chi.csv&amp;time=20200404-0917'
resource_url = f'https://api.data.gov.hk/v1/historical-archive/get-file?url={s}'
df_v1 = pd.read_csv(resource_url)
df_v1[-5:]

,個案編號,報告日期,發病日期,性別,年齡,入住醫院名稱,住院/出院/死亡,香港/非香港居民,個案分類*,確診/疑似個案
841,842,03/04/2020,02/04/2020,男,20,北區醫院,住院,香港居民,本地個案的密切接觸者,確診
842,843,03/04/2020,02/04/2020,女,19,威爾斯親王醫院,住院,香港居民,輸入個案,確診
843,844,03/04/2020,02/04/2020,女,59,威爾斯親王醫院,住院,香港居民,本地個案的密切接觸者,確診
844,845,03/04/2020,31/03/2020,女,47,伊利沙伯醫院,住院,香港居民,輸入個案,確診
845,846,03/04/2020,24/03/2020,男,51,待定,待入院,香港居民,本地個案的密切接觸者,確診


### Latest schema (v2)

Note the last two columns definition have been changed

In [3]:
# Get the latest data
resource_url = 'http://www.chp.gov.hk/files/misc/enhanced_sur_covid_19_chi.csv'
df_v2 = pd.read_csv(resource_url)
df_v2[-5:]

,個案編號,報告日期,發病日期,性別,年齡,入住醫院名稱,住院/出院/死亡,香港/非香港居民,分類*,個案狀況*
15436,97093,06/02/2022,待定,待定,待定,NaN,待入院,待定,本地感染個案,待定
15437,97094,06/02/2022,待定,待定,待定,NaN,待入院,待定,本地感染個案,待定
15438,97095,06/02/2022,待定,待定,待定,NaN,待入院,待定,本地感染個案,待定
15439,97096,06/02/2022,待定,女,72,NaN,待入院,待定,本地感染個案,待定
15440,97097,06/02/2022,待定,女,11,NaN,待入院,待定,本地感染個案,待定


### Notice the severe delay 數據更新嚴重滯後

Although the datasource was claimed to update every night, we can see a very severe delay in the data report dates.  For examples, on the 8 Mar 2022, the API only return the patient list upto 6 Feb 2022 !

<strong> A total of 4 weeks delay !🤯 </strong>

<strong>有數據但係delay四個星期! 🤯 </strong>


## Lets inspect the unique values in each column

In [10]:
PREVIEW_COLS = ['性別','年齡','入住醫院名稱','住院/出院/死亡','香港/非香港居民','分類*','個案狀況*','個案分類*','確診/疑似個案']
def preview(df):
    print('-' * 20, "Columns", '-'* 20)
    print("columns", ":\n", list(df.columns.values))

    print('-' * 20, "Distinct Values", '-'* 20)
    for col in PREVIEW_COLS:
        if col in df.columns.values:
            print(col, ":\n" , df[col].unique())
    
    print('-' * 20, "Dates Example", '-'* 20)
    for col in df[['報告日期', '發病日期']]:
        print(col, ":\n" , df[col][0:5].values)
        
#preview(df_v1)
preview(df_v2)

-------------------- Columns --------------------
columns :
 ['個案編號', '報告日期', '發病日期', '性別', '年齡', '入住醫院名稱', '住院/出院/死亡', '香港/非香港居民', '分類*', '個案狀況*']
-------------------- Distinct Values --------------------
性別 :
 ['男' '女' nan '待定']
年齡 :
 ['39' '56' '62' '63' '47' '68' '64' '73' '72' '37' '75' '80' '60' '25'
 '28' '55' '65' '58' '42' '24' '70' '91' '57' '22' '50' '51' '23' '69'
 '86' '52' '59' '71' '66' '43' '67' '41' '61' '54' '45' '46' '32' '83'
 '38' '48' '96' '76' '35' '33' '49' '21' '16' '29' '26' '89' '79' '84'
 '44' '36' '31' '1' '30' '18' '27' '34' '20' '12' '19' '40' '4' '13' '17'
 '15' '11' '74' '78' '7' '14' '77' '53' '<1' '2' '93' '6' '3' '8' '5' '10'
 '9' '85' '92' '95' '87' '90' '94' '82' '88' '81' '97' '100' '98' nan '待定']
入住醫院名稱 :
 [nan]
住院/出院/死亡 :
 ['出院' '死亡' '不適用' nan '住院' '待入院']
香港/非香港居民 :
 ['非香港居民' '香港居民' nan '待定']
分類* :
 ['輸入個案' '與輸入個案有流行病學關連' '可能本地個案' '本地個案' '與本地個案有流行病學關連' '與可能本地個案有流行病學關連' nan
 '本地感染個案']
個案狀況* :
 ['確診' '删除' '無症狀感染' '復陽' '待定' '未知']
------------------

## Data cleansing

There are quite some inconsistent values/typos we need to clean.
Here is a quick & dirty cleaning by mapping all invalid values to NaN.
I also simplified the import/local cases category.

In [11]:
# Parse date
def parse_ddmmYYYY(s):
    try:
        if isinstance(s, str):
            if s in ['無病徵','不適用','未知', '一月','三月中','七月中','十月' , 'Unknown']:
                return np.NaN
            else:
                return datetime.datetime.strptime(s,'%d/%m/%Y')
        else:
            return np.NaN
    except:
        print(f"****** Failed to parse ddmmYYY: {s}, type={type(s)} ******")
        return np.NaN

# Parse Age
def parse_age(s):
    try:
        if isinstance(s, str):
            if s == '<1' or '個月' in s:
                return float(0)
            elif s in ['不適用', 'pending']:
                return np.NaN
            else:
                return float(s)
        elif math.isnan(s):
            return np.NaN
        else:
            return float(s)
    except:
        print(f"****** Failed to parse Age: {s}, type={type(s)} ******")
        return np.NaN

# Parse Category
def parse_category(s):
    if isinstance(s, str):
        if '輸入個案' in s:
            return '輸入個案'
        elif '本地' in s:
            return '本地個案'
        else:
            return s
    else:
        return np.NaN

# Clean the data
def cleanse(df):
    df = df.copy()
    
    # Map V1 to V2 cols
    if '個案分類*' in df.columns.values:
        df.rename(columns={'個案分類*':'分類*'}, inplace=True)
    if '確診/疑似個案' in df.columns.values:
        df.rename(columns={'確診/疑似個案':'個案狀況*'}, inplace=True)
    
    # Parsing and Defaults
    df = df.replace(['待定'], np.NaN)
    df['年齡'] = df['年齡'].apply(parse_age)    
    df['發病日期'] = df['發病日期'].apply(parse_ddmmYYYY)
    df['報告日期'] = df['報告日期'].apply(parse_ddmmYYYY)
    df['分類*'] = df['分類*'].apply(parse_category)
    
    # Compute
    try:
        df['報告與發病日差'] = (df['報告日期'] - df['發病日期']).dt.days
    except:
        print(f"**Failed to compute 報告與發病日差")
        df['報告與發病日差'] = np.NaN
    return df

df = cleanse(df_v2)

## Statistics

Now we can play around with different grouping statistics. For examples:

### Total Count and Average Age of each category 

In [6]:
from IPython.display import display

display(
    df.groupby(
        ['分類*', '住院/出院/死亡', '個案狀況*'] #'性別'
    ).agg({
        '性別': ['count'],
        '年齡': ['mean']
    }).round(1).replace(np.NaN, '-')
)
print("From ", min(df['報告日期']), " to ", max(df['報告日期']))

性別    年齡
                     count  mean
分類*  住院/出院/死亡 個案狀況*             
本地個案 住院       無症狀感染     82  48.4
              確診       816  45.0
     出院       復陽         1  29.0
              無症狀感染    149  37.2
              確診     10103  45.2
     待入院      無症狀感染     28  35.7
              確診       117  37.9
     死亡       確診       207  78.5
輸入個案 不適用      確診        36  37.8
     住院       無症狀感染     22  42.2
              確診        24  38.1
     出院       復陽         9  30.9
              無症狀感染    240  36.3
              確診      3552  34.8
     待入院      未知         1  41.0
              無症狀感染      6  40.8
     死亡       確診         6  60.7

From  2020-01-23 00:00:00  to  2022-02-06 00:00:00


### 第一至第四波疫情死亡人數共213人

In [7]:
start_date = '2019-01-01'
end_date = '2021-12-31'

display(
    df.where(
       (df['報告日期'] >= start_date) & (df['報告日期'] < end_date)
    ).groupby(
        ['住院/出院/死亡']
    ).agg({
        '性別': ['count'],
        '年齡': ['mean']
    }).round(1).replace(np.NaN, '-')
)

,性別,年齡
,count,mean
住院/出院/死亡,,
不適用,36,37.8
出院,12382,43.1
死亡,213,78.0


### 核對一下衞生署衞生防護中心資料:

截至2022年3月7日，衞生署衞生防護中心累計報告2287宗死亡個案。香港在2021年12月下旬開始出現第五波疫情，自第五波疫情起，共錄得2074宗死亡個案。因此第一至第四波疫情死亡人數是 2287 - 2074 = 213. So our code seems correct !

<br>

#  Part 2 - Estimate hospitalized time and waiting time 住院日數及等候時間

來看更複雜的細節，我地可以分析每位患者的時間線，估算患者的：
- 等待入院日數
- 住院日數

例如:  待入院 => 住院 => 出院
中間的時間相差應該就是 "等待入院日數" 及 "住院日數"

Now we come to a more sophisticated analysis: tracking the timeline of each patient. The goal is to estimate:
- the waiting time to admit to hospital
- the staying time inside the hosptial


### STEP1:  download all historical snapshot CSV to local

<strong>This steps can take a long time.</strong>
And it seems wget is having a better performance than python. So the code below will generate a shell script full of wget commands, then run the script itself.

In [12]:
# To reduce network bandwidth, we download once and save locally
data_dir = './raw'
resource_name = 'enhanced_sur_covid_19_chi.csv'
resource_url = f'http://www.chp.gov.hk/files/misc/{resource_name}'

YESTERDAY = datetime.datetime.now() - datetime.timedelta(1)
DAY_1 = datetime.date(2020,1,1)
DAY_N = datetime.date(2024,1,1)

os.makedirs(data_dir, exist_ok=True)

def get_historical_timestamps(resource_url, start_date, end_date):
    start = start_date.strftime('%Y%m%d')
    end = end_date.strftime('%Y%m%d')
    query = f"https://api.data.gov.hk/v1/historical-archive/list-file-versions?url={resource_url}&amp;start={start}&amp;end={end}"
    j = requests.get(query).json()
    return dict(zip(j['data-dictionary-dates'],j['timestamps']))

def generate_download_script():
    os.makedirs(data_dir, exist_ok=True)
    download_script = open(Path(data_dir,"download.sh"), "w")
    timestamps = get_historical_timestamps(resource_url, DAY_1, YESTERDAY)
    download_script.write('#!/bin/sh\n')
    for yyyymmdd, file_timestamp in timestamps.items():
        query = f"https://api.data.gov.hk/v1/historical-archive/get-file?url={resource_url}&amp;time={file_timestamp}"
        output_file = f'{resource_name}_{yyyymmdd}'
        download_script.write(f"if [ ! -f {output_file} ]; then \n")
        download_script.write(f'   echo Downloading "{output_file}" from "{query}" \n')
        download_script.write(f'   wget "{query}" -q -O "{output_file}" \n')
        download_script.write(f"fi\n")
        #urllib.request.urlretrieve(query, f'{resource_name}_{yyyymmdd}')
    print("Generated:", download_script)
    download_script.close()

# Kick off the download
generate_download_script()
os.system(f"cd {data_dir}; sh ./download.sh")


Generated: <_io.TextIOWrapper name='raw/download.sh' mode='w' encoding='UTF-8'>


0

### STEP2: Read all CSV into dataframe, cleanse them and stack them together

<strong>This steps can take a long time.</strong>

In [13]:
def concate_history():
    history = None
    for i, date in enumerate(pd.date_range(DAY_1, YESTERDAY)):
        cached_df = Path(data_dir, f"{resource_name}_{date.strftime('%Y%m%d')}_cleansed")
        if cached_df.exists():
            print(f"Processing {date}  ... cached:", cached_df)
            df = pd.read_pickle(cached_df)
        else:
            print(f"Processing {date}  ...")
            cached_csv = Path(data_dir, f"{resource_name}_{date.strftime('%Y%m%d')}")
            if not cached_csv.exists():
                #print(f"Warning: {cached_csv} is missing")
                continue
            df = pd.read_csv(cached_csv)
            df = cleanse(df)
            df.to_pickle(cached_df)
        # day1
        if history is None:
            history = df
        # Concate
        df['record_date'] = date
        history = history.append(df)
    return history

history = concate_history()

Processing 2020-01-01 00:00:00  ...
Processing 2020-01-02 00:00:00  ...
Processing 2020-01-03 00:00:00  ...
Processing 2020-01-04 00:00:00  ...
Processing 2020-01-05 00:00:00  ...
Processing 2020-01-06 00:00:00  ...
Processing 2020-01-07 00:00:00  ...
Processing 2020-01-08 00:00:00  ...
Processing 2020-01-09 00:00:00  ...
Processing 2020-01-10 00:00:00  ...
Processing 2020-01-11 00:00:00  ...
Processing 2020-01-12 00:00:00  ...
Processing 2020-01-13 00:00:00  ...
Processing 2020-01-14 00:00:00  ...
Processing 2020-01-15 00:00:00  ...
Processing 2020-01-16 00:00:00  ...
Processing 2020-01-17 00:00:00  ...
Processing 2020-01-18 00:00:00  ...
Processing 2020-01-19 00:00:00  ...
Processing 2020-01-20 00:00:00  ...
Processing 2020-01-21 00:00:00  ...
Processing 2020-01-22 00:00:00  ...
Processing 2020-01-23 00:00:00  ...
Processing 2020-01-24 00:00:00  ...
Processing 2020-01-25 00:00:00  ...
Processing 2020-01-26 00:00:00  ...
Processing 2020-01-27 00:00:00  ...
Processing 2020-01-28 00:00:

Processing 2020-06-17 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200617_cleansed
Processing 2020-06-18 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200618_cleansed
Processing 2020-06-19 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200619_cleansed
Processing 2020-06-20 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200620_cleansed
Processing 2020-06-21 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200621_cleansed
Processing 2020-06-22 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200622_cleansed
Processing 2020-06-23 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200623_cleansed
Processing 2020-06-24 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200624_cleansed
Processing 2020-06-25 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200625_cleansed
Processing 2020-06-26 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200626_cleansed
Processing 2020-06-27 00:00:00  ... cach

Processing 2020-09-15 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200915_cleansed
Processing 2020-09-16 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200916_cleansed
Processing 2020-09-17 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200917_cleansed
Processing 2020-09-18 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200918_cleansed
Processing 2020-09-19 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200919_cleansed
Processing 2020-09-20 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200920_cleansed
Processing 2020-09-21 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200921_cleansed
Processing 2020-09-22 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200922_cleansed
Processing 2020-09-23 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200923_cleansed
Processing 2020-09-24 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20200924_cleansed
Processing 2020-09-25 00:00:00  ... cach

Processing 2020-12-11 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201211_cleansed
Processing 2020-12-12 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201212_cleansed
Processing 2020-12-13 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201213_cleansed
Processing 2020-12-14 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201214_cleansed
Processing 2020-12-15 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201215_cleansed
Processing 2020-12-16 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201216_cleansed
Processing 2020-12-17 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201217_cleansed
Processing 2020-12-18 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201218_cleansed
Processing 2020-12-19 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201219_cleansed
Processing 2020-12-20 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20201220_cleansed
Processing 2020-12-21 00:00:00  ... cach

Processing 2021-03-07 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210307_cleansed
Processing 2021-03-08 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210308_cleansed
Processing 2021-03-09 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210309_cleansed
Processing 2021-03-10 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210310_cleansed
Processing 2021-03-11 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210311_cleansed
Processing 2021-03-12 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210312_cleansed
Processing 2021-03-13 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210313_cleansed
Processing 2021-03-14 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210314_cleansed
Processing 2021-03-15 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210315_cleansed
Processing 2021-03-16 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210316_cleansed
Processing 2021-03-17 00:00:00  ... cach

Processing 2021-06-02 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210602_cleansed
Processing 2021-06-03 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210603_cleansed
Processing 2021-06-04 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210604_cleansed
Processing 2021-06-05 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210605_cleansed
Processing 2021-06-06 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210606_cleansed
Processing 2021-06-07 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210607_cleansed
Processing 2021-06-08 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210608_cleansed
Processing 2021-06-09 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210609_cleansed
Processing 2021-06-10 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210610_cleansed
Processing 2021-06-11 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210611_cleansed
Processing 2021-06-12 00:00:00  ... cach

Processing 2021-08-27 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210827_cleansed
Processing 2021-08-28 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210828_cleansed
Processing 2021-08-29 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210829_cleansed
Processing 2021-08-30 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210830_cleansed
Processing 2021-08-31 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210831_cleansed
Processing 2021-09-01 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210901_cleansed
Processing 2021-09-02 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210902_cleansed
Processing 2021-09-03 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210903_cleansed
Processing 2021-09-04 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210904_cleansed
Processing 2021-09-05 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20210905_cleansed
Processing 2021-09-06 00:00:00  ... cach

Processing 2021-11-21 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211121_cleansed
Processing 2021-11-22 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211122_cleansed
Processing 2021-11-23 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211123_cleansed
Processing 2021-11-24 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211124_cleansed
Processing 2021-11-25 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211125_cleansed
Processing 2021-11-26 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211126_cleansed
Processing 2021-11-27 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211127_cleansed
Processing 2021-11-28 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211128_cleansed
Processing 2021-11-29 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211129_cleansed
Processing 2021-11-30 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20211130_cleansed
Processing 2021-12-01 00:00:00  ... cach

Processing 2022-02-15 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220215_cleansed
Processing 2022-02-16 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220216_cleansed
Processing 2022-02-17 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220217_cleansed
Processing 2022-02-18 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220218_cleansed
Processing 2022-02-19 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220219_cleansed
Processing 2022-02-20 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220220_cleansed
Processing 2022-02-21 00:00:00  ... cached: raw/enhanced_sur_covid_19_chi.csv_20220221_cleansed
Processing 2022-02-22 00:00:00  ...
Processing 2022-02-23 00:00:00  ...
Processing 2022-02-24 00:00:00  ...
Processing 2022-02-25 00:00:00  ...
Processing 2022-02-26 00:00:00  ...
Processing 2022-02-27 00:00:00  ...
Processing 2022-02-28 00:00:00  ...
Processing 2022-03-01 00:00:00  ...
Processing 2022-03-02 00:00:00  ...
Proc

### STEP3: Combine the timeline of each patient

<Strong>Again this part takes time</Strong>

In [18]:
def merge_history(history):
    H = history
    case_ids = H['個案編號'].unique()
    extra_cols = ['待入院|有待提供@日期', '住院@日期', '出院@日期','死亡@日期','不適用|nan@日期']

    df_merged = pd.DataFrame()
    for i, case_id in enumerate(case_ids):
        if i % 1000 == 0:
            print(f"Merging the {i}th case ...")
        patient = H[H['個案編號'] == case_id]
        pending_date = patient[(patient['住院/出院/死亡'] == '待入院') | (patient['住院/出院/死亡'] == '有待提供')].record_date.min()
        enter_date = patient[patient['住院/出院/死亡'] == '住院'].record_date.min()
        exit_date = patient[patient['住院/出院/死亡'] == '出院'].record_date.min()
        pass_date = patient[patient['住院/出院/死亡'] == '死亡'].record_date.min()
        #na_date = patient[(patient['住院/出院/死亡'] == '不適用') | (patient['住院/出院/死亡'].isnull()) ].record_date.min()

        latest = patient[-1:].copy()
        latest['待入院|有待提供@日期'] = pending_date
        latest['住院@日期'] = enter_date
        latest['出院@日期'] = exit_date
        latest['死亡@日期'] = pass_date
        #latest['不適用|nan@日期'] = na_date

        try:
            latest['輪候日數'] = latest['住院@日期'] - latest['待入院|有待提供@日期']
        except:
            latest['輪候日數'] = np.NaN

        try:
            latest['住院日數'] = latest['出院@日期'] - latest['住院@日期']
        except:
            latest['住院日數'] = np.NaN
        df_merged = df_merged.append(latest)

    # Convert dt to days
    df_merged['輪候日數'] = df_merged['輪候日數'].dt.days
    df_merged['住院日數'] = df_merged['住院日數'].dt.days
    return df_merged

df_merged = merge_history(history)

Merging the 0th case ...
Merging the 1000th case ...
Merging the 2000th case ...
Merging the 3000th case ...
Merging the 4000th case ...
Merging the 5000th case ...
Merging the 6000th case ...
Merging the 7000th case ...
Merging the 8000th case ...
Merging the 9000th case ...
Merging the 10000th case ...
Merging the 11000th case ...
Merging the 12000th case ...
Merging the 13000th case ...
Merging the 14000th case ...
Merging the 15000th case ...


### STEP4: Add age, year, month and week bins 

In [19]:
def cut_bins(df_merged):
    # Age Bins
    bins_age = [0,2,5,11,18,25,35,45,55,65,75,85,150]
    bins_age_label = ['0-1 歲','2-4 歲','5-10 歲','11-17 歲','18-24 歲','25-34 歲',
            '35-44 歲','45-54 歲','55-64 歲','65-74 歲','75-84 歲','85 歲以上']

    # Time Bins
    tz= None #"Asia/Hong_Kong" #Not needed
    YEARS = [year for year in pd.date_range(DAY_1, DAY_N, freq='YS', tz=tz)]
    bins_year = [np.datetime64(d) for d in YEARS]
    bins_year_label = [ d.year for d in YEARS[:-1]]

    MONTHS = [month for month in pd.date_range(DAY_1, DAY_N, freq='MS', tz=tz)]
    bins_month = [np.datetime64(d) for d in MONTHS]
    bins_month_label = [ f'{d.month}/{d.year}' for d in MONTHS[:-1]]

    WEEKS = [week for week in pd.date_range(DAY_1, DAY_N, freq='W-SUN', tz=tz)]
    bins_week = [np.datetime64(d) for d in WEEKS]
    bins_week_label = [ f'{d.day}/{d.month}/{d.year}' for d in WEEKS[:-1]]

    df = df_merged.copy()
    df['年齡組別'] = pd.cut(df['年齡'], bins=bins_age, labels=bins_age_label, right=False)
    df['年'] = pd.cut(df['報告日期'], bins=bins_year, labels=bins_year_label, right=False)
    df['月'] = pd.cut(df['報告日期'], bins=bins_month, labels=bins_month_label, right=False)
    df['週'] = pd.cut(df['報告日期'], bins=bins_week, labels=bins_week_label, right=False)
    return df

df = cut_bins(df_merged)

Lets save the result after all these long program run:

In [20]:
df.to_pickle("df.pickle")
df = pd.read_pickle("df.pickle")

### Group by age group 以年齡組別分類

In [31]:
# Analyze Period
start_date = np.datetime64(datetime.date(2020,1,1))
end_date = np.datetime64(datetime.date(2022,10,1))
pd.set_option('display.max_rows', 4000)

# Query
groups = df[
    (
        (df['住院/出院/死亡'] == '出院')|
        (df['住院/出院/死亡'] == '死亡')|
        (df['住院/出院/死亡'] == '住院')|
        (df['住院/出院/死亡'] == '待入院')
    )&
    (df['報告日期'] >= start_date) &
    (df['報告日期'] <= end_date)
].groupby(
    ['年', '年齡組別']
).agg({
    '性別': ['count'],
    '輪候日數': ['mean'],
    '住院日數': ['mean', 'sum']
}).round(1).replace(np.NaN, '-')

g = groups[
    groups['性別']['count'] > 0
]
display(g)

性別 輪候日數  住院日數         
             count mean  mean      sum
年    年齡組別                             
2020 0-1 歲      61  1.0  11.2    638.0
     2-4 歲     109  1.2  10.8   1167.0
     5-10 歲    188  1.2  10.5   1947.0
     11-17 歲   305  1.2  11.7   3490.0
     18-24 歲   800  1.4  11.5   8748.0
     25-34 歲  1340  1.3  10.9  14021.0
     35-44 歲  1439  1.3  11.1  15221.0
     45-54 歲  1413  1.3  11.3  15482.0
     55-64 歲  1582  1.2  13.9  20946.0
     65-74 歲   999  1.1  18.3  17155.0
     75-84 歲   387  1.1  24.4   7569.0
     85 歲以上    194  1.0  27.5   3632.0
2021 0-1 歲      75  1.0  12.6    921.0
     2-4 歲      67  1.0  11.4    766.0
     5-10 歲    100  1.0  11.7   1156.0
     11-17 歲   136  1.0  12.4   1649.0
     18-24 歲   304  1.0  11.5   3493.0
     25-34 歲   847  1.0  11.0   8981.0
     35-44 歲   888  1.0  10.3   8910.0
     45-54 歲   543  1.0  12.5   6600.0
     55-64 歲   433  1.0  12.5   5323.0
     65-74 歲   259  1.0  16.8   4196.0
     75-84 歲   108  1.0  22.0   2130.0
     85 歲以上     37  1.0  28.4    738.0
2022 0-1 歲      38  1.2  16.1    242.0
     2-4 歲      54  1.4  12.1    291.0
     5-10 歲    125  1.4  10.5    703.0
     11-17 歲   185  1.3  11.1   1419.0
     18-24 歲   263  1.4  10.5   2263.0
     25-34 歲   482  1.5  10.5   3595.0
     35-44 歲   507  1.6  10.1   3222.0
     45-54 歲   476  1.5  11.2   3165.0
     55-64 歲   355  1.4  12.3   1853.0
     65-74 歲   186  1.4  12.6    996.0
     75-84 歲    80  1.0  12.7    317.0
     85 歲以上     27  1.5  18.5     74.0

### Group by year  以每年計

In [29]:
# Analyze Period
start_date = np.datetime64(datetime.date(2020,1,1))
end_date = np.datetime64(datetime.date(2022,10,1))

# Query
groups = df[
    (
        (df['住院/出院/死亡'] == '出院')|
        (df['住院/出院/死亡'] == '死亡')|
        (df['住院/出院/死亡'] == '住院')|
        (df['住院/出院/死亡'] == '待入院')
    )&
    (df['報告日期'] >= start_date) &
    (df['報告日期'] <= end_date)
].groupby(
    ['年','月']
).agg({
    '性別': ['count'],
    '輪候日數': ['mean'],
    '住院日數': ['mean', 'sum']
}).round(1).replace(np.NaN, '-')


pd.set_option('display.max_rows', 4000)
g = groups[
    groups['性別']['count'] > 0
]
display(g)

性別 輪候日數  住院日數         
             count mean  mean      sum
年    月                                
2020 1/2020     13    -     -      0.0
     2/2020     82    -  30.3     91.0
     3/2020    620    -  17.6   9395.0
     4/2020    323  1.0  17.1   5517.0
     5/2020     47  1.0  19.0    893.0
     6/2020    121  1.0  14.4   1699.0
     7/2020   2040  1.8  12.7  24847.0
     8/2020   1537  1.3  12.0  17868.0
     9/2020    277  1.0  12.8   3313.0
     10/2020   234  1.0  11.4   2401.0
     11/2020   991  1.0  12.7  12103.0
     12/2020  2532  1.0  13.1  31889.0
2021 1/2021   1605  1.0  12.5  19350.0
     2/2021    553  1.0  10.3   5428.0
     3/2021    459  1.0  10.7   4775.0
     4/2021    305  1.0  10.9   3193.0
     5/2021     67  1.0   6.6    430.0
     6/2021     79  1.0  11.7    915.0
     7/2021     61  1.5  10.6    644.0
     8/2021    126  1.0  11.4   1431.0
     9/2021    110  1.0   9.5   1040.0
     10/2021   129  1.0  13.6   1754.0
     11/2021    90  1.0  17.8   1606.0
     12/2021   213  1.0  20.4   4297.0
2022 1/2022   1554  1.3  12.5  16660.0
     2/2022   1225  1.7   4.6   1480.0

### Group by week 以每週計

In [33]:
# Analyze Period
start_date = np.datetime64(datetime.date(2020,1,1))
end_date = np.datetime64(datetime.date(2022,10,1))

# Query
groups = df[
    (
        (df['住院/出院/死亡'] == '出院')|
        (df['住院/出院/死亡'] == '死亡')|
        (df['住院/出院/死亡'] == '住院')|
        (df['住院/出院/死亡'] == '待入院')
    )&
    (df['報告日期'] >= start_date) &
    (df['報告日期'] <= end_date)
].groupby(
    ['月', '週']
).agg({
    '性別': ['count'],
    '輪候日數': ['mean'],
    '住院日數': ['mean', 'sum']
}).round(1).replace(np.NaN, '-')


pd.set_option('display.max_rows', 4000)
g = groups[
    groups['性別']['count'] > 0
]
display(g)

性別 輪候日數  住院日數        
                   count mean  mean     sum
月       週                                  
1/2020  19/1/2020      5    -     -     0.0
        26/1/2020      8    -     -     0.0
2/2020  26/1/2020      2    -     -     0.0
        2/2/2020      11    -     -     0.0
        9/2/2020      30    -  29.5    59.0
        16/2/2020     14    -  32.0    32.0
        23/2/2020     25    -     -     0.0
3/2020  1/3/2020      15    -  41.7   125.0
        8/3/2020      32    -  19.7   296.0
        15/3/2020    132    -  13.4  1423.0
        22/3/2020    309    -  17.8  5010.0
        29/3/2020    132    -  19.9  2541.0
4/2020  29/3/2020    148  1.0  18.3  2713.0
        5/4/2020     138  1.0  16.5  2280.0
        12/4/2020     23  1.0  15.0   344.0
        19/4/2020     14    -  12.9   180.0
5/2020  26/4/2020      2    -  16.0    32.0
        3/5/2020       5    -  18.2    91.0
        10/5/2020      8  1.0  14.9   119.0
        17/5/2020     13    -  18.8   244.0
        24/5/2020     17    -  20.1   341.0
        31/5/2020      2    -  33.0    66.0
6/2020  31/5/2020     21  1.0  17.5   333.0
        7/6/2020       4    -  16.0    64.0
        14/6/2020     19  1.0  14.7   279.0
        21/6/2020     69  1.0  13.3   903.0
        28/6/2020      8  1.0  15.0   120.0
7/2020  28/6/2020     53  1.0  12.8   677.0
        5/7/2020     168  1.0  18.3  2907.0
        12/7/2020    344  1.5  16.9  5591.0
        19/7/2020    719  1.5  12.6  8845.0
        26/7/2020    756  2.1   9.5  6827.0
8/2020  26/7/2020    125  1.6  10.3  1244.0
        2/8/2020     611  1.5  11.2  6529.0
        9/8/2020     398  1.1  12.5  4849.0
        16/8/2020    251  1.0  13.0  3202.0
        23/8/2020    128  1.0  13.8  1683.0
        30/8/2020     24  1.0  15.7   361.0
9/2020  30/8/2020     47  1.0  12.6   518.0
        6/9/2020      81  1.0  12.5   951.0
        13/9/2020     71  1.0  15.0  1032.0
        20/9/2020     50  1.0  11.0   539.0
        27/9/2020     28  1.0  11.4   273.0
10/2020 27/9/2020     21  1.0   9.8   197.0
        4/10/2020     67  1.0  14.0   843.0
        11/10/2020    62  1.0   9.4   543.0
        18/10/2020    50  1.0  11.3   488.0
        25/10/2020    34  1.0  11.0   330.0
11/2020 1/11/2020     41  1.0  10.5   367.0
        8/11/2020     80  1.0  10.5   770.0
        15/11/2020   116  1.0  12.8  1397.0
        22/11/2020   563  1.0  12.9  7126.0
        29/11/2020   191  1.0  13.1  2443.0
12/2020 29/11/2020   488  1.0  15.4  7345.0
        6/12/2020    644  1.0  12.8  7947.0
        13/12/2020   632  1.0  13.1  7950.0
        20/12/2020   462  1.0  11.6  5062.0
        27/12/2020   306  1.0  12.2  3585.0
1/2021  27/12/2020    77  1.0  12.1   882.0
        3/1/2021     288  1.0  12.3  3346.0
        10/1/2021    290  1.0  11.6  3226.0
        17/1/2021    507  1.0  14.1  6938.0
        24/1/2021    390  1.0  11.7  4407.0
        31/1/2021     53  1.0  10.6   551.0
2/2021  31/1/2021    156  1.0  10.3  1547.0
        7/2/2021     159  1.0  10.1  1558.0
        14/2/2021     81  1.0   8.4   608.0
        21/2/2021    135  1.0  11.4  1494.0
        28/2/2021     22  1.0  10.5   221.0
3/2021  28/2/2021     69  1.0  10.7   727.0
        7/3/2021     182  1.0  11.2  2021.0
        14/3/2021    112  1.0  10.5  1147.0
        21/3/2021     74  1.0   9.8   656.0
        28/3/2021     22  1.0  10.7   224.0
4/2021  28/3/2021     34  1.0  10.5   336.0
        4/4/2021      67  1.0  11.4   738.0
        11/4/2021     85  1.0  10.3   835.0
        18/4/2021     75  1.0   9.9   732.0
        25/4/2021     44  1.0  13.1   552.0
5/2021  25/4/2021      8  1.0   4.8    38.0
        2/5/2021      23  1.0   8.3   190.0
        9/5/2021      15  1.0   6.6    99.0
        16/5/2021     10  1.0   3.8    34.0
        23/5/2021      7  1.0   8.1    57.0
        30/5/2021      4  1.0   4.0    12.0
6/2021  30/5/2021     10  1.1   9.9    99.0
        6/6/2021      26  1.0  10.3   269.0
        13/6/2021      7  1.0   7.1    50.0
        20/6/2021 